In [ ]:
suppressPackageStartupMessages({
    source(".Rprofile")
    library(Seurat)
    library(Signac)
})

In [ ]:
PATH <- "e02_cca_anchor"
if (!dir.exists(PATH)) {
    dir.create(PATH)
}

# Read data

In [ ]:
rna <- read_h5ad("e01_preprocessing/rna_agg.h5ad")

In [ ]:
atac <- read_h5ad("e01_preprocessing/atac_agg.h5ad")

In [ ]:
atac2rna <- read_h5ad("e01_preprocessing/atac2rna_agg.h5ad")

In [ ]:
rna.so <- CreateSeuratObject(
    counts = Matrix::t(rna$X), assay = "RNA", meta.data = rna$obs
)

In [ ]:
atac.so <- CreateSeuratObject(
    counts = Matrix::t(atac$X), assay = "ATAC", meta.data = atac$obs
)

In [ ]:
atac.so[["ACTIVITY"]] <- CreateAssayObject(counts = Matrix::t(atac2rna$X))

# Preprocessing

## RNA

In [ ]:
rna.so <- NormalizeData(rna.so)

In [ ]:
VariableFeatures(rna.so) <- rownames(rna$var)[rna$var$highly_variable]

In [ ]:
rna.so <- ScaleData(rna.so)

In [ ]:
rna.so <- RunPCA(rna.so, seed.use = 0, verbose = FALSE)

## ATAC2RNA

In [ ]:
DefaultAssay(atac.so) <- "ACTIVITY"

In [ ]:
VariableFeatures(atac.so) <- rownames(rna$var)[rna$var$highly_variable]

In [ ]:
atac.so <- NormalizeData(atac.so)

In [ ]:
atac.so <- ScaleData(atac.so)

## ATAC

In [ ]:
DefaultAssay(atac.so) <- "ATAC"

In [ ]:
atac.so <- RunTFIDF(atac.so)

In [ ]:
atac.so <- FindTopFeatures(atac.so, min.cutoff = "q0")

In [ ]:
atac.so <- RunSVD(atac.so)

In [ ]:
rm(rna, atac, atac2rna)

# Integration

In [ ]:
transfer.anchors <- FindTransferAnchors(
    reference = rna.so, query = atac.so,
    features = VariableFeatures(rna.so),
    reference.assay = "RNA", query.assay = "ACTIVITY",
    reduction = "cca"
)

In [ ]:
refdata <- GetAssayData(
    rna.so, assay = "RNA", slot = "data"
)[VariableFeatures(rna.so), ]

In [ ]:
imputation <- TransferData(
    anchorset = transfer.anchors, refdata = refdata,
    weight.reduction = atac.so[["lsi"]], dims = 1:30
)

In [ ]:
atac.so[["RNA"]] <- imputation

In [ ]:
coembed.so <- merge(x = rna.so, y = atac.so)
coembed.so <- ScaleData(
    coembed.so, features = VariableFeatures(rna.so), do.scale = FALSE
)
coembed.so <- RunPCA(
    coembed.so, features = VariableFeatures(rna.so),
    seed.use = 0, verbose = FALSE
)

# Save results

In [ ]:
combined_latent <- Embeddings(coembed.so)

In [ ]:
rna_latent <- combined_latent[colnames(rna.so), ]

In [ ]:
atac_latent <- combined_latent[colnames(atac.so), ]

In [ ]:
write.table(
    rna_latent, file.path(PATH, "rna_latent.csv"),
    sep = ",", row.names = TRUE, col.names = FALSE, quote = FALSE
)

In [ ]:
write.table(
    atac_latent, file.path(PATH, "atac_latent.csv"),
    sep = ",", row.names = TRUE, col.names = FALSE, quote = FALSE
)